## Amazon AgentCore Bedrock 코드 인터프리터 튜토리얼

이 튜토리얼에서는 AgentCore Bedrock 코드 인터프리터를 사용하여 다음 작업을 수행하는 방법을 보여줍니다.
1. 샌드박스 환경 설정
2. 데이터 로드 및 분석
3. 샌드박스 환경에서 코드 실행
4. 결과 처리 및 검색

## 필수 조건
- Bedrock AgentCore 코드 인터프리터 액세스 권한이 있는 AWS 계정
- 코드 인터프리터 리소스를 생성하고 관리하는 데 필요한 IAM 권한이 있어야 합니다.
- 필요한 Python 패키지 설치(boto3 및 bedrock-agentcore 포함)
- 샘플 데이터 파일(data.csv)
- 분석 스크립트(stats.py)

## IAM 실행 역할에 다음 IAM 정책이 연결되어 있어야 합니다.

~~~ {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateCodeInterpreter",
                "bedrock-agentcore:StartCodeInterpreterSession",
                "bedrock-agentcore:InvokeCodeInterpreter",
                "bedrock-agentcore:StopCodeInterpreterSession",
                "bedrock-agentcore:DeleteCodeInterpreter",
                "bedrock-agentcore:ListCodeInterpreters",
                "bedrock-agentcore:GetCodeInterpreter"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
        }
    ]
}



## 작동 방식

코드 실행 샌드박스는 코드 인터프리터, 셸, 파일 시스템을 갖춘 격리된 환경을 생성하여 에이전트가 사용자 쿼리를 안전하게 처리할 수 있도록 합니다. 대규모 언어 모델(LLM)을 통해 도구 선택을 돕고, 코드는 이 세션 내에서 실행된 후 합성을 위해 사용자 또는 에이전트에게 반환됩니다.

![architecture local](code-interpreter.png)

## 1. 환경 설정

먼저, 필요한 라이브러리를 가져오고 코드 인터프리터 클라이언트를 초기화해 보겠습니다.

In [1]:
!pip install --upgrade -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 180.0 MB/s  0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.1
    Uninstalling botocore-1.37.1:
      Successfully uninstalled botocore-1.37.1
  Attempting uninstall: s3transfer━━━━━━━━━━━━━━ 0/4 [botocore]
    Found existing installation: s3transfer 0.11.30/4 [botocore]
    Uninstalling s3transfer-0.11.3:━━━━━━━━━ 0/4 [botocore]
      Successfully uninstalled s3transfer-0.11.3 0/4 [botocore]
  Attempting uninstall: boto3━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [s3transfer]
    Found existing installation: boto3 1.37.1━━━━━━━━━━━━━━━━━ 1/4 [s3transfer]
    Uninstalling boto3-1.37.1:90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [s3transfer]
      Successfully uninstalled boto3-1.37.1━━━━━━━━━━━━━━━━━━━ 1/4 [s3transfer]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [bedrock-agentcore]ansfer]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [2]:
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
import json
from typing import Dict, Any, List

# Initialize the Code Interpreter with your AWS region
code_client = CodeInterpreter('us-west-2')
code_client.start()

'01K403MS2RWN7PNT3YXAT58PP9'

## 2. 로컬 파일 읽기

이제 샘플 데이터 파일과 분석 스크립트의 내용을 읽어 보겠습니다.

In [3]:
def read_file(file_path: str) -> str:
    """Helper function to read file content with error handling"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return ""
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

# Read both files
data_file_content = read_file("samples/data.csv")
code_file_content = read_file("samples/stats.py")

## 3. 샌드박스 환경을 위한 파일 준비

샌드박스 환경에 생성할 파일을 정의하는 구조체를 생성합니다.

In [4]:
files_to_create = [
    {
        "path": "data.csv",
        "text": data_file_content
    },
    {
        "path": "stats.py",
        "text": code_file_content
    }
]

## 4. 도구 호출을 위한 도우미 함수 생성

이 도우미 함수를 사용하면 샌드박스 도구를 더 쉽게 호출하고 응답을 처리할 수 있습니다. 활성 세션 내에서 지원되는 언어(Python, JavaScript)로 코드를 실행하고, 종속성 구성에 따라 라이브러리에 액세스하고, 시각화를 생성하고, 실행 간 상태를 유지할 수 있습니다.

In [5]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Helper function to invoke sandbox tools
    
    Args:
        tool_name (str): Name of the tool to invoke
        arguments (Dict[str, Any]): Arguments to pass to the tool
        
    Returns:
        Dict[str, Any]: JSON formatted result
    """
    response = code_client.invoke(tool_name, arguments)
    for event in response["stream"]:
        return json.dumps(event["result"])

## 5. 샌드박스에 파일 쓰기

이제 샌드박스 환경에 파일을 쓰고 파일이 성공적으로 생성되었는지 확인해 보겠습니다.

In [6]:
# Write files to sandbox
writing_files = call_tool("writeFiles", {"content": files_to_create})
print("Writing files result:")
print(writing_files)

# Verify files were created
listing_files = call_tool("listFiles", {"path": ""})
print("\nFiles in sandbox:")
print(listing_files)

Writing files result:
{"content": [{"type": "text", "text": "Successfully wrote all 2 files"}], "isError": false}

Files in sandbox:
{"content": [{"type": "resource_link", "uri": "file:///log", "name": "log", "description": "Directory"}, {"type": "resource_link", "uri": "file:///.ipython", "name": ".ipython", "description": "Directory"}, {"type": "resource_link", "mimeType": "text/x-python", "uri": "file:///stats.py", "name": "stats.py", "description": "File"}, {"type": "resource_link", "mimeType": "text/csv", "uri": "file:///data.csv", "name": "data.csv", "description": "File"}], "isError": false}


## 6. 분석 실행

이제 샌드박스 환경에서 분석 스크립트를 실행하고 결과를 처리해 보겠습니다.

In [7]:
import pprint

# Execute the analysis script
code_execute_result = call_tool("executeCode", {
    "code": files_to_create[1]['text'],
    "language": "python",
    "clearContext": True
})

# Parse and display results
analysis_results = json.loads(code_execute_result)
print("Full analysis results:")
pprint.pprint(analysis_results)

print("\nStandard output from analysis:")
print(analysis_results['structuredContent']['stdout'])

Full analysis results:
{'content': [{'text': 'Name Preferred_City Preferred_Animal Preferred_Thing\n'
                      'count       299130         299130           '
                      '299130          299130\n'
                      'unique        1722             55               '
                      '50              51\n'
                      'top     Lisa White         Prague             '
                      'Goat          Pencil\n'
                      'freq           222           5587             '
                      '6141            6058',
              'type': 'text'}],
 'isError': False,
 'structuredContent': {'executionTime': 1.0435175895690918,
                       'exitCode': 0,
                       'stderr': '',
                       'stdout': 'Name Preferred_City Preferred_Animal '
                                 'Preferred_Thing\n'
                                 'count       299130         299130           '
                                 '2

## 7. 정리

마지막으로, 코드 인터프리터 세션을 종료하여 정리합니다. 세션 사용이 끝나면 세션을 종료하여 리소스를 해제하고 불필요한 요금 발생을 방지해야 합니다.

In [8]:
# Stop the Code Interpreter session
code_client.stop()
print("Code Interpreter session stopped successfully!")

Code Interpreter session stopped successfully!


## 결론

이 튜토리얼에서는 다음 방법을 학습했습니다.
- 코드 인터프리터 세션 초기화
- 분석을 위해 파일 읽기 및 준비
- 샌드박스 환경에서 코드 실행
- 결과 처리 및 표시
- 리소스 정리